<a href="https://colab.research.google.com/github/fertorresfs/Pipeline-Para-Validacao-Automatizada-de-Dados/blob/main/pipeline_para_validacao_automatizada_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import logging
from datetime import datetime

# Configurar o logging
logging.basicConfig(filename=f'validacao_dados_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def validar_dados(funcao):
    """Decorador para registrar informações de validação."""
    def wrapper(df, *args, **kwargs):
        nome_funcao = funcao.__name__
        logging.info(f"Executando validação: {nome_funcao}")
        resultado = funcao(df, *args, **kwargs)
        logging.info(f"Validação {nome_funcao}: {'Passou' if resultado else 'Falhou'}")
        return resultado
    return wrapper

In [ ]:
@validar_dados
def verificar_nulos(df, colunas):
    """Verifica se há valores nulos nas colunas especificadas."""
    for col in colunas:
        if df[col].isnull().any():
            logging.warning(f"Coluna '{col}' contém valores nulos.")
            return False # retorna falso se houver algum nulo
    return True

In [ ]:
@validar_dados
def verificar_unicos(df, colunas):
    """Verifica se os valores em uma coluna são únicos."""
    for col in colunas:
        if len(df[col].unique()) != len(df[col]): #conferindo se o número de valores únicos é igual ao tamanho da coluna
            logging.warning(f"Coluna '{col}' contém valores duplicados.")
            return False
    return True

In [ ]:
@validar_dados
def verificar_intervalo(df, coluna, min_val, max_val):
    """Verifica se os valores em uma coluna estão dentro de um intervalo."""

    if not (df[coluna] >= min_val).all() or not (df[coluna] <= max_val).all(): #.all para verificar em todo o df
        logging.warning(f"Coluna '{coluna}' contém valores fora do intervalo [{min_val}, {max_val}].")
        return False
    return True

In [ ]:
@validar_dados
def verificar_duplicatas(df):
    """Verifica se há linhas duplicadas no DataFrame."""
    if df.duplicated().any():
        logging.warning("O DataFrame contém linhas duplicadas.")
        return False
    return True


In [ ]:
def executar_validacao(df, validacoes):
    """Executa as validações especificadas."""

    resultados = {}
    for validacao in validacoes:
        func = validacao['funcao']
        args = validacao.get('args', [])
        kwargs = validacao.get('kwargs', {})
        resultados[func.__name__] = func(df, *args, **kwargs)


    return resultados

In [ ]:
# Exemplo de uso:
dados = {
    'id': [1, 2, 3, 4, 5, 5], #id duplicado
    'nome': ['Alice', 'Bob', 'Charlie', 'David', None], #nome nulo
    'idade': [25, 30, 22, 45, 28],
    'valor': [100, 200, 150, 500, 10000] #outlier
}

df = pd.DataFrame(dados)

In [ ]:
validacoes = [
    {'funcao': verificar_nulos, 'args': [['nome']]},
    {'funcao': verificar_unicos, 'args': [['id']]},
    {'funcao': verificar_intervalo, 'args': ['idade'], 'kwargs': {'min_val': 18, 'max_val': 65}},
    {'funcao': verificar_duplicatas}
]

In [ ]:
resultados_validacao = executar_validacao(df, validacoes)

In [ ]:
print(resultados_validacao) #exibe no console
# Mostra no log o resultado de cada validação (Passou ou Falhou)